In [1]:
import xarray as xr
import dask

#ds = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr')
ds = xr.open_zarr('gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr')
ds

<xarray.Dataset> Size: 89TB
Dimensions:                                           (time: 93544,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 13)
Coordinates:
  * latitude                                          (latitude) float32 3kB ...
  * level                                             (level) int64 104B 50 ....
  * longitude                                         (longitude) float32 6kB ...
  * time                                              (time) datetime64[ns] 748kB ...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_wind_speed                                    (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    above_ground                                      (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, latitude, longitude) float32 388GB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    vorticity                                         (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    wind_speed                                        (time, level, latitude, longitude) float32 5TB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>

In [2]:
lat_data = ds.latitude
lat_bounds = slice(55, 47) if lat_data[0] > lat_data[-1] else slice(47, 55)

lon_data = ds.longitude
lon_bounds = slice(15, 6) if lon_data[0] > lon_data[-1] else slice(6, 15)




In [3]:
temperature_at850hpa = ds.temperature.sel(level=1000, 
                                          time='2010-01',
                                          latitude=lat_bounds,
                                          longitude=lon_bounds
                                          )
temperature_at850hpa



<xarray.DataArray 'temperature' (time: 124, latitude: 33, longitude: 37)> Size: 606kB
dask.array<getitem, shape=(124, 33, 37), dtype=float32, chunksize=(1, 33, 37), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 132B 55.0 54.75 54.5 54.25 ... 47.5 47.25 47.0
    level      int64 8B 50
  * longitude  (longitude) float32 148B 6.0 6.25 6.5 6.75 ... 14.5 14.75 15.0
  * time       (time) datetime64[ns] 992B 2010-01-01 ... 2010-01-31T18:00:00
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

In [4]:
# Aggregate the data into larger tiles. Here assuming that each tile is represented by a 2x2 grid.
# You need to adjust '2' based on the resolution of your dataset and the size of the new tiles you desire.
temperature_larger_tiles = temperature_at850hpa.coarsen(
    latitude=8, 
    longitude=12,
    boundary='trim'  # 'trim' drops any data not fitting into the window perfectly; use 'pad' to include all data.
).mean()


temperature_larger_tiles = temperature_larger_tiles.compute()

In [5]:
#df = temperature_at850hpa.to_dataframe()
df = temperature_larger_tiles.to_dataframe()
print(df.shape)
print(df)

# one time point:
# 1h resolution: 13 sec
# 6h resolution: 5 sec

# one day:
# 1h resolution: 4min
# 6h resolutioin: 15s

# one month:
# 1h
# 6h resolution: 7 min

(1488, 2)
                                        level  temperature
time                latitude longitude                    
2010-01-01 00:00:00 54.125   7.375         50   213.089035
                             10.375        50   212.044739
                             13.375        50   211.105713
                    52.125   7.375         50   214.530273
                             10.375        50   213.747314
...                                       ...          ...
2010-01-31 18:00:00 50.125   10.375        50   202.332275
                             13.375        50   203.491455
                    48.125   7.375         50   202.279129
                             10.375        50   204.449631
                             13.375        50   206.596313

[1488 rows x 2 columns]


In [6]:
print(df.head(30))

                                        level  temperature
time                latitude longitude                    
2010-01-01 00:00:00 54.125   7.375         50   213.089035
                             10.375        50   212.044739
                             13.375        50   211.105713
                    52.125   7.375         50   214.530273
                             10.375        50   213.747314
                             13.375        50   213.643631
                    50.125   7.375         50   215.199539
                             10.375        50   215.122665
                             13.375        50   214.798752
                    48.125   7.375         50   216.788986
                             10.375        50   215.792557
                             13.375        50   214.137527
2010-01-01 06:00:00 54.125   7.375         50   211.151001
                             10.375        50   210.636765
                             13.375        50   210.4517

In [7]:
print(df.head(20))

                                        level  temperature
time                latitude longitude                    
2010-01-01 00:00:00 54.125   7.375         50   213.089035
                             10.375        50   212.044739
                             13.375        50   211.105713
                    52.125   7.375         50   214.530273
                             10.375        50   213.747314
                             13.375        50   213.643631
                    50.125   7.375         50   215.199539
                             10.375        50   215.122665
                             13.375        50   214.798752
                    48.125   7.375         50   216.788986
                             10.375        50   215.792557
                             13.375        50   214.137527
2010-01-01 06:00:00 54.125   7.375         50   211.151001
                             10.375        50   210.636765
                             13.375        50   210.4517